In [35]:
import pandas as pd
import numpy as np
import yfinance as yf
import requests
from bs4 import BeautifulSoup
import quandl

In [ ]:
def read_data(excel_name, skipped_rows=4):
    df = pd.read_excel(excel_name, skiprows = 4, index_col=0)
    df = df.dropna(how='all')
    df = df.dropna(axis=1, how='all')
    return_df = df.T
    return_df.index.name = 'Year'
    
    return_df.index = pd.to_datetime(return_df.index, format='%m/%d/%Y')    
    recorded_dates = return_df.index
    return_df.index = return_df.index.year
    return_df = return_df.sort_index()    
    
    return return_df, recorded_dates

business_cycle_df, recorded_dates = read_data("Data/Business Cycle Indicators (23).xlsx")
consumer_credit_df, _ = read_data("Data/Consumer Credit (23).xlsx")
consumer_prices_df, _ = read_data("Data/Consumer Prices (23).xlsx")
employment_cost_df, _ = read_data("Data/Employment Cost Index (23).xlsx")
employment_sit_df, _ = read_data("Data/Employment Situation (23).xlsx")
housing_market_df, _ = read_data("Data/Housing Market (23).xlsx")
industrial_prod_df, _ = read_data("Data/Industrial Production (23).xlsx")
personal_consumption_df, _ = read_data("Data/Personal Consumption Expenditures (23).xlsx")
retail_sales_df, _ = read_data("Data/Retail Sales (23).xlsx")

In [3]:
master_df = pd.concat([business_cycle_df, consumer_credit_df, consumer_prices_df, employment_cost_df, employment_sit_df, housing_market_df, industrial_prod_df, personal_consumption_df, retail_sales_df], axis=1)
# master_df

Now we can download tickers of the 500 S&P stocks

In [4]:
import requests
from bs4 import BeautifulSoup

def extract_tickers(url):
    try:
        # Fetch the page
        response = requests.get(url)
        response.raise_for_status()  # Ensure the response is successful

        # Parse the page with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all table row elements
        rows = soup.find_all('tr')

        # List to store ticker symbols
        tickers = []

        # Extract ticker symbols from each row
        for row in rows:
            # Each ticker symbol is in the first 'td' in each 'row' within an 'a' tag with class 'external text'
            ticker_link = row.find('a', class_='external text')
            if ticker_link:
                tickers.append(ticker_link.text)

        return tickers
    except requests.RequestException as e:
        print(f"An error occurred while fetching the page: {e}")
        return []
    except Exception as e:
        print(f"An error occurred during parsing: {e}")
        return []

In [21]:
def get_yearly_closing_prices(tickers, start_year=None, end_year=None):
    results = {}
    for ticker in tickers:
        try:
            # Construct the date range for data fetching
            start_date = f"{start_year}-01-01" if start_year else "1900-01-01"
            end_date = f"{end_year}-12-31" if end_year else "today"

            # Fetch historical data for the stock within the specified date range
            data = yf.download(ticker, start=start_date, end=end_date, progress=False)

            # Check if data is empty
            if data.empty:
                print(f"No data found for ticker {ticker}. Skipping...")
                continue

            # Resample the data to get the last day of each year
            yearly_data = data['Adj Close'].resample('Y').last()

            # Reset index to make 'Date' a column and rename columns appropriately
            yearly_data = yearly_data.reset_index()
            yearly_data.columns = ['Year', 'Closing Price']

            # Extract only the year part from the datetime object for better readability
            yearly_data['Year'] = yearly_data['Year'].dt.year

            # Store the result in the dictionary
            results[ticker] = yearly_data
        except Exception as e:
            print(f"An error occurred with ticker {ticker}: {e}")
            continue

    return results

In [22]:
# URL of the Wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

# Extract the tickers from the Wikipedia page
tickers = extract_tickers(url)

In [23]:
data = get_yearly_closing_prices(tickers, 2004, 2023)


1 Failed download:
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')


No data found for ticker BRK.B. Skipping...



1 Failed download:
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2004-01-01 -> 2023-12-31)')


No data found for ticker BF.B. Skipping...



1 Failed download:
['GEV']: Exception("%ticker%: Data doesn't exist for startDate = 1072933200, endDate = 1703998800")


No data found for ticker GEV. Skipping...



1 Failed download:
['SOLV']: Exception("%ticker%: Data doesn't exist for startDate = 1072933200, endDate = 1703998800")


No data found for ticker SOLV. Skipping...


In [31]:
closing_prices_df = pd.concat(data.values(), keys=data.keys(), names=['Ticker']).reset_index(level=0)

# make the ticker column into the column headers, and the year into the index
closing_prices_df.index = closing_prices_df['Year']
closing_prices_df = closing_prices_df.pivot(columns='Ticker', values='Closing Price')
closing_prices_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Year,,,,,,,,,,,,,,,,,,,,,
2004,14.598245,NaN,0.973590,NaN,NaN,14.220021,4.300000,19.303823,31.363848,23.427706,...,69.978523,12.804468,36.554527,8.875668,27.040377,NaN,11.896439,70.197853,56.279999,NaN
2005,20.164963,35.013420,2.173644,NaN,NaN,12.302720,6.083333,20.890678,36.959999,22.984585,...,64.327377,13.010035,29.961378,9.425516,30.227436,NaN,11.923183,59.088150,42.849998,NaN
2006,22.404455,50.766628,2.565196,NaN,NaN,15.607718,7.512222,27.019629,41.119999,21.455666,...,71.002563,14.340691,54.754971,12.302523,42.040249,NaN,15.112262,68.673325,34.790001,NaN
2007,23.619511,13.867724,5.989087,NaN,NaN,18.423845,7.816667,26.638918,42.730000,21.130796,...,69.578629,15.459943,68.570320,12.542002,52.250919,NaN,19.996271,57.957905,34.700001,NaN
2008,10.048258,7.287391,2.580616,NaN,NaN,17.965921,7.788889,24.629204,21.290001,13.088793,...,47.195076,6.719945,25.843065,10.814759,45.402092,NaN,16.789339,35.414341,20.260000,NaN
2009,19.974361,4.562868,6.371570,NaN,NaN,18.788601,7.950000,31.776018,36.779999,22.466343,...,52.113140,9.650451,37.817097,13.004604,39.679169,NaN,19.092579,51.789745,28.350000,NaN
2010,26.634630,9.436841,9.752823,NaN,NaN,17.234993,9.783333,37.826279,30.780001,27.565762,...,70.677307,11.555391,73.214874,15.096876,43.698246,NaN,27.354944,47.032200,37.990002,NaN
2011,22.455900,4.779698,12.245457,NaN,NaN,20.988131,12.410000,42.362709,28.270000,26.900465,...,81.378006,11.747349,82.065025,18.465126,51.855690,21.771255,33.579739,46.804405,35.779999,NaN
2012,26.582815,12.727012,16.233421,NaN,NaN,25.247692,14.673333,54.118023,37.680000,32.620682,...,72.529221,17.983225,90.799995,18.553940,54.293587,23.335766,38.492428,59.053822,39.310001,NaN


In [33]:
# merge the closing prices with the master_df
final_df = pd.concat([master_df, closing_prices_df], axis=1)
final_df

,Composite index of 10 leading indicators (2016=100),Change from previous month (pct.),"Change over 6-month span (AR, pct.)","Diffusion index, 1-mo. span (pct.)","Diffusion index, 6-mo. span (pct.)",Composite index of 4 coincident indicators (2016=100),Change from previous month (pct.),"Change over 6-month span (AR, pct.)","Diffusion index, 1-mo. span (pct.)","Diffusion index, 6-mo. span (pct.)",...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Year,,,,,,,,,,,,,,,,,,,,,
2004,114.599998,-0.3,-7.7,50.0,20.0,109.599998,0.5,1.7,100.0,100.0,...,69.978523,12.804468,36.554527,8.875668,27.040377,NaN,11.896439,70.197853,56.279999,NaN
2005,113.900002,-0.6,-7.8,50.0,40.0,109.900002,0.3,2.0,100.0,75.0,...,64.327377,13.010035,29.961378,9.425516,30.227436,NaN,11.923183,59.088150,42.849998,NaN
2006,112.900002,-0.9,-7.7,30.0,40.0,109.800003,-0.1,1.8,25.0,75.0,...,71.002563,14.340691,54.754971,12.302523,42.040249,NaN,15.112262,68.673325,34.790001,NaN
2007,111.800003,-1.0,-8.4,20.0,10.0,109.599998,-0.2,1.1,25.0,75.0,...,69.578629,15.459943,68.570320,12.542002,52.250919,NaN,19.996271,57.957905,34.700001,NaN
2008,111.0,-0.7,-9.4,40.0,20.0,109.599998,0.0,0.7,62.5,50.0,...,47.195076,6.719945,25.843065,10.814759,45.402092,NaN,16.789339,35.414341,20.260000,NaN
2009,110.400002,-0.5,-9.2,50.0,10.0,110.099998,0.5,1.1,100.0,50.0,...,52.113140,9.650451,37.817097,13.004604,39.679169,NaN,19.092579,51.789745,28.350000,NaN
2010,109.699997,-0.6,-8.6,30.0,30.0,110.199997,0.1,1.8,62.5,75.0,...,70.677307,11.555391,73.214874,15.096876,43.698246,NaN,27.354944,47.032200,37.990002,NaN
2011,108.400002,-1.2,-8.5,10.0,40.0,110.300003,0.1,1.6,75.0,75.0,...,81.378006,11.747349,82.065025,18.465126,51.855690,21.771255,33.579739,46.804405,35.779999,NaN
2012,107.599998,-0.7,-8.0,30.0,30.0,110.400002,0.1,1.6,62.5,100.0,...,72.529221,17.983225,90.799995,18.553940,54.293587,23.335766,38.492428,59.053822,39.310001,NaN


Now we get Futures Data

In [43]:
def fetch_annual_futures_data(futures_contracts, start_year=2004, end_year=2023):
    quandl.ApiConfig.api_key = 'K9yLEyBxEgsvUhnipN8L'

    annual_futures_data = {}

    start_date = f"{start_year}-01-01"
    end_date = f"{end_year}-12-31"

    for name, code in futures_contracts.items():
        try:
            data = quandl.get(code, start_date=start_date, end_date=end_date)
            annual_data = data['Settle'].resample('Y').last().reset_index()
            annual_data.rename(columns={'Settle': 'Annual Closing Price', 'Date': 'Year'}, inplace=True)
            annual_data['Year'] = annual_data['Year'].dt.year
            annual_futures_data[name] = annual_data
            print(f"Annual closing data for {name} downloaded successfully.")
        except Exception as e:
            print(f"Failed to download data for {name}: {e}")

    return annual_futures_data

futures_contracts = {
    'Crude_Oil_WTI': "CHRIS/CME_CL1",
    'Gold': "CHRIS/CME_GC1",
    'Silver': "CHRIS/CME_SI1",
    'Corn': "CHRIS/CME_C1",
    'Soybeans': "CHRIS/CME_S1",
    'Wheat': "CHRIS/CME_W1",
    'UST_10Y': "CHRIS/CME_TY1",
    'Eurodollar': "CHRIS/CME_ED1",
    'Euro_FX': "CHRIS/CME_EC1",
    'Japanese_Yen': "CHRIS/CME_JY1",
    'British_Pound': "CHRIS/CME_BP1",
    'S&P_500': "CHRIS/CME_SP1",
    'NASDAQ_100': "CHRIS/CME_NQ1",
    'Dow_Jones_Industrial': "CHRIS/CME_YM1"
}

futures_data = fetch_annual_futures_data(futures_contracts, 2004, 2023)

Annual closing data for Crude_Oil_WTI downloaded successfully.
Annual closing data for Gold downloaded successfully.
Annual closing data for Silver downloaded successfully.
Annual closing data for Corn downloaded successfully.
Annual closing data for Soybeans downloaded successfully.
Annual closing data for Wheat downloaded successfully.
Annual closing data for UST_10Y downloaded successfully.
Annual closing data for Eurodollar downloaded successfully.
Annual closing data for Euro_FX downloaded successfully.
Annual closing data for Japanese_Yen downloaded successfully.
Annual closing data for British_Pound downloaded successfully.
Annual closing data for S&P_500 downloaded successfully.
Annual closing data for NASDAQ_100 downloaded successfully.
Annual closing data for Dow_Jones_Industrial downloaded successfully.


In [44]:
futures_data_df = pd.concat(futures_data.values(), keys=futures_data.keys(), names=['Futures Contract']).reset_index(level=0)
futures_data_df.index = futures_data_df['Year']
futures_data_df = futures_data_df.pivot(columns='Futures Contract', values='Annual Closing Price')
futures_data_df

Futures Contract,British_Pound,Corn,Crude_Oil_WTI,Dow_Jones_Industrial,Euro_FX,Eurodollar,Gold,Japanese_Yen,NASDAQ_100,S&P_500,Silver,Soybeans,UST_10Y,Wheat
Year,,,,,,,,,,,,,,
2004,1.9071,204.75,43.45,NaN,1.35580,97.095,438.4,9797.0,1628.50,1213.7,7.160,547.75,111.937500,320.75
2005,1.7187,215.75,61.04,NaN,1.18800,95.225,518.9,8551.0,1659.00,1254.8,8.960,602.00,109.406250,339.25
2006,1.9585,390.25,61.05,NaN,1.32410,94.680,638.0,8484.0,1775.00,1428.4,12.820,683.50,107.468750,501.00
2007,1.9785,455.50,95.98,13328.00,1.45900,95.765,838.0,9013.0,2104.75,1477.2,14.797,1199.00,113.390625,885.00
2008,1.4557,407.00,44.60,8727.00,1.39210,98.940,884.3,11029.0,1212.50,900.1,11.270,972.25,125.750000,610.75
2009,1.6146,414.50,79.36,10365.00,1.43340,99.645,1096.2,10740.0,1858.75,1110.7,16.822,1039.75,115.453125,541.50
2010,1.5581,629.00,91.38,11513.00,1.33640,99.635,1421.4,12328.0,2216.00,1253.0,30.910,1393.75,120.437500,794.25
2011,1.5500,646.50,98.83,12150.00,1.29680,99.355,1566.8,13010.0,2274.75,1252.6,27.875,1198.50,131.125000,652.75
2012,1.6240,698.25,91.82,13027.00,1.32080,99.700,1675.8,11550.0,2659.25,1420.1,30.173,1418.75,132.781250,778.00


In [45]:
# merge the futures data with the final_df
final_df = pd.concat([final_df, futures_data_df], axis=1)
final_df

,Composite index of 10 leading indicators (2016=100),Change from previous month (pct.),"Change over 6-month span (AR, pct.)","Diffusion index, 1-mo. span (pct.)","Diffusion index, 6-mo. span (pct.)",Composite index of 4 coincident indicators (2016=100),Change from previous month (pct.),"Change over 6-month span (AR, pct.)","Diffusion index, 1-mo. span (pct.)","Diffusion index, 6-mo. span (pct.)",...,Euro_FX,Eurodollar,Gold,Japanese_Yen,NASDAQ_100,S&P_500,Silver,Soybeans,UST_10Y,Wheat
Year,,,,,,,,,,,,,,,,,,,,,
2004,114.599998,-0.3,-7.7,50.0,20.0,109.599998,0.5,1.7,100.0,100.0,...,1.35580,97.095,438.4,9797.0,1628.50,1213.7,7.160,547.75,111.937500,320.75
2005,113.900002,-0.6,-7.8,50.0,40.0,109.900002,0.3,2.0,100.0,75.0,...,1.18800,95.225,518.9,8551.0,1659.00,1254.8,8.960,602.00,109.406250,339.25
2006,112.900002,-0.9,-7.7,30.0,40.0,109.800003,-0.1,1.8,25.0,75.0,...,1.32410,94.680,638.0,8484.0,1775.00,1428.4,12.820,683.50,107.468750,501.00
2007,111.800003,-1.0,-8.4,20.0,10.0,109.599998,-0.2,1.1,25.0,75.0,...,1.45900,95.765,838.0,9013.0,2104.75,1477.2,14.797,1199.00,113.390625,885.00
2008,111.0,-0.7,-9.4,40.0,20.0,109.599998,0.0,0.7,62.5,50.0,...,1.39210,98.940,884.3,11029.0,1212.50,900.1,11.270,972.25,125.750000,610.75
2009,110.400002,-0.5,-9.2,50.0,10.0,110.099998,0.5,1.1,100.0,50.0,...,1.43340,99.645,1096.2,10740.0,1858.75,1110.7,16.822,1039.75,115.453125,541.50
2010,109.699997,-0.6,-8.6,30.0,30.0,110.199997,0.1,1.8,62.5,75.0,...,1.33640,99.635,1421.4,12328.0,2216.00,1253.0,30.910,1393.75,120.437500,794.25
2011,108.400002,-1.2,-8.5,10.0,40.0,110.300003,0.1,1.6,75.0,75.0,...,1.29680,99.355,1566.8,13010.0,2274.75,1252.6,27.875,1198.50,131.125000,652.75
2012,107.599998,-0.7,-8.0,30.0,30.0,110.400002,0.1,1.6,62.5,100.0,...,1.32080,99.700,1675.8,11550.0,2659.25,1420.1,30.173,1418.75,132.781250,778.00


In [46]:
final_df.to_csv('final_df.csv')